In [18]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import os
os.environ["ZE_ENABLE_TRACING_LAYER"] = "1"

import torch
import torch.nn as nn

print(f"PyTorch Version: {torch.__version__}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

PyTorch Version: 2.4.1+cu121
Using device: cuda


importar simple tokenizer.py, model.py

In [20]:
import sys
import os

# Obtém o caminho absoluto para o diretório 'src'
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))

# Adiciona o diretório 'src' ao sys.path
if src_path not in sys.path:
    sys.path.append(src_path)

# Agora você pode importar módulos de dentro de 'src' como se fosse um pacote de nível superior

In [21]:
import clip

In [22]:
clip_model, preprocess = clip.load("ViT-B/32", device=device)

In [23]:
import argparse
import numpy as np
import random

In [24]:
parser = argparse.ArgumentParser()
parser.add_argument('--raf_path', type=str, default='../../data/raf-basic', help='raf_dataset_path')
parser.add_argument('--resnet50_path', type=str, default='../../data/resnet50_ft_weight.pkl', help='pretrained_backbone_path')
parser.add_argument('--label_path', type=str, default='list_patition_label.txt', help='label_path')
parser.add_argument('--workers', type=int, default=2, help='number of workers')
parser.add_argument('--batch_size', type=int, default=32, help='batch_size')
parser.add_argument('--w', type=int, default=7, help='width of the attention map')
parser.add_argument('--h', type=int, default=7, help='height of the attention map')
parser.add_argument('--gpu', type=int, default=0, help='the number of the device')
parser.add_argument('--lam', type=float, default=5, help='kl_lambda')
parser.add_argument('--epochs', type=int, default=60, help='number of epochs')
args = parser.parse_args(args=[])

### Arquitetura do Modelo

In [25]:
import cafe.model
from cafe.model import Model

### Treinamento E Teste Codigos

In [26]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [27]:
from tqdm import tqdm

def train(model, train_loader, optimizer, scheduler, device):
  running_loss = 0.0
  iter_cnt = 0
  correct_sum = 0

  model.to(device)
  model.train()

  total_loss = []
  with tqdm(total=len(train_loader)) as pbar:
      for batch_i, (imgs1, labels) in enumerate(train_loader):
        imgs1 = imgs1.to(device)
        labels = labels.to(device)

        criterion = nn.CrossEntropyLoss(reduction='none')

        output, MC_loss = model(imgs1, clip_model, labels, phase='train')

        loss1 = nn.CrossEntropyLoss()(output, labels)

        loss = loss1 + 5 * MC_loss[1] + 1.5 * MC_loss[0]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        iter_cnt += 1
        _, predicts = torch.max(output, 1)
        correct_num = torch.eq(predicts, labels).sum()
        correct_sum += correct_num
        running_loss += loss

        pbar.update(1)  # Update progress bar for each batch

  scheduler.step()
  running_loss = running_loss / iter_cnt
  acc = correct_sum.float() / float(train_loader.dataset.__len__())
  return acc, running_loss

setup_seed(3407)

In [28]:
def test(model, test_loader, device):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        iter_cnt = 0
        correct_sum = 0
        data_num = 0


        for batch_i, (imgs1, labels) in enumerate(test_loader):
            imgs1 = imgs1.to(device)
            labels = labels.to(device)


            outputs, _ = model(imgs1, clip_model, labels, phase='test')


            loss = nn.CrossEntropyLoss()(outputs, labels)

            iter_cnt += 1
            _, predicts = torch.max(outputs, 1)

            correct_num = torch.eq(predicts, labels).sum()
            correct_sum += correct_num

            running_loss += loss
            data_num += outputs.size(0)

        running_loss = running_loss / iter_cnt
        test_acc = correct_sum.float() / float(data_num)
        
    return test_acc, running_loss

In [29]:
def load_pretrained_weights(model, checkpoint):
    import collections
    if 'state_dict' in checkpoint:
        state_dict = checkpoint['state_dict']
    else:
        state_dict = checkpoint
    model_dict = model.state_dict()
    new_state_dict = collections.OrderedDict()
    matched_layers, discarded_layers = [], []
    for k, v in state_dict.items():
        # If the pretrained state_dict was saved as nn.DataParallel,
        # keys would contain "module.", which should be ignored.
        if k.startswith('module.'):
            k = k[7:]
        if k in model_dict and model_dict[k].size() == v.size():
            new_state_dict[k] = v
            matched_layers.append(k)
        else:
            discarded_layers.append(k)
    # new_state_dict.requires_grad = False
    model_dict.update(new_state_dict)

    model.load_state_dict(model_dict)
    print('load_weight', len(matched_layers))
    return model

In [30]:
# Caminho do modelo em relação ao diretório atual (onde está o notebook)
model_path = os.path.join(os.path.dirname(os.getcwd()), 'models', 'resnet18_msceleb.pth')

# Carregando o modelo
model = Model(model_path=model_path)
model.to(device)

weight : torch.Size([7, 512])
bias : torch.Size([7])


Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [31]:
import torch.optim as optim
from torch.optim.lr_scheduler import ExponentialLR

optimizer = optim.Adam(model.parameters(), lr=0.0002, weight_decay=0.0001)
scheduler = ExponentialLR(optimizer, gamma=0.9)

In [32]:
from torchvision import transforms

In [33]:
eval_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

In [34]:
train_transforms = transforms.Compose([
    #transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomErasing(scale=(0.02, 0.25))
    ])

### Caregamento de Dataset

In [35]:
save_path = os.path.join(os.path.dirname(os.getcwd()), 'models/cafe')
# os.makedirs() com exist_ok=True é a forma mais concisa
os.makedirs(save_path, exist_ok=True)
print(f"Diretório '{save_path}' garantido (criado se não existia).")

Diretório '/home/sergiojunior/FER-generalizavel/models/cafe' garantido (criado se não existia).


# Teste cruzado

### Download de datasets

In [36]:
import kagglehub

# Download latest version
path_rafdb = kagglehub.dataset_download("shuvoalok/raf-db-dataset")

print("Path to dataset files:", path_rafdb)

Path to dataset files: /home/sergiojunior/.cache/kagglehub/datasets/shuvoalok/raf-db-dataset/versions/2


In [37]:
import kagglehub

# Download latest version
path_fer = kagglehub.dataset_download("arnabkumarroy02/ferplus")

print("Path to dataset files:", path_fer)

Path to dataset files: /home/sergiojunior/.cache/kagglehub/datasets/arnabkumarroy02/ferplus/versions/3


In [38]:
import kagglehub

# Download latest version
path_mma = kagglehub.dataset_download("mahmoudima/mma-facial-expression")
path_mma = path_mma + '/MMAFEDB'
print("Path to dataset files:", path_mma)

Path to dataset files: /home/sergiojunior/.cache/kagglehub/datasets/mahmoudima/mma-facial-expression/versions/1/MMAFEDB


### Teste entre dominios

In [42]:
from aux.emotion_idx import *
from emotion_datasets import MMADataset, FERPlusDataset, RafDataSet, SFEWDataset, AffectNetDataset

In [43]:
temp_affectnet_root = '/workspace/sergiojunior/AffectNet-8Labels'

In [61]:
"""
dataset_sfew = SFEWDataset(root_dir=path_sfew, idx_test=emotion_to_index_raf, idx_sfew=index_to_emotion_sfew, split="Val", transform=eval_transforms)
loader_sfew = torch.utils.data.DataLoader(dataset_sfew, batch_size=args.batch_size,
                                          shuffle=False,
                                          num_workers=args.workers,
                                          pin_memory=True)
"""

dataset_mma = MMADataset(root_dir=path_mma, idx_test=emotion_to_index_raf, idx_mma=index_to_emotion_mma, split="test", transform=eval_transforms)
loader_mma = torch.utils.data.DataLoader(dataset_mma, batch_size=args.batch_size,
                                          shuffle=False,
                                          num_workers=args.workers,
                                          pin_memory=True)

dataset_affect = AffectNetDataset(root_dir=temp_affectnet_root, idxs_test=emotion_to_index_raf, idxs_aff=index_to_emotion_aff, subset='val', transform=eval_transforms)
loader_affect = torch.utils.data.DataLoader(dataset_affect, batch_size=args.batch_size,
                                          shuffle=False,
                                          num_workers=args.workers,
                                          pin_memory=True)

dataset_raf = RafDataSet(path_rafdb, idxs_test=emotion_to_index_raf, idxs_raf=index_to_emotion_raf, train=False, transform=eval_transforms)
loader_raf = torch.utils.data.DataLoader(dataset_raf,
                                           batch_size=args.batch_size,
                                           shuffle=False,
                                           num_workers=args.workers,
                                           pin_memory=True)

dataset_fer = FERPlusDataset(root_dir=path_fer, idxs_test=emotion_to_index_raf, idxs_fer=index_to_emotion_fer, subset="test", transform=eval_transforms)
loader_fer = torch.utils.data.DataLoader(dataset_fer, batch_size=args.batch_size,
                                          shuffle=False,
                                          num_workers=args.workers,
                                          pin_memory=True)

Carregado 3500 itens do val_set.
3093


In [62]:
checkpoint = torch.load(f"{save_path}/ours_best_RAFDB.pth")
checkpoint = checkpoint["model_state_dict"]
model = load_pretrained_weights(model, checkpoint)
model.to(device)

load_weight 122


<ipython-input-62-9fc62cc829c2>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(f"{save_path}/ours_best_RAFDB.pth")


Model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [63]:
###### SFEW
acc_raf_sfew, test_loss = test(model, loader_sfew, device)
print('test acc dbtrain-raf dbtest-sfew: ', acc_raf_sfew)

NameError: name 'loader_sfew' is not defined

In [64]:

###### AFFECT
acc_raf_affect, test_loss = test(model, loader_affect, device)
print('test acc dbtrain-raf dbtest-affect: ', acc_raf_affect)

### RAFDB
acc_raf_raf, test_loss = test(model, loader_raf, device)
print('test acc dbtrain-raf dbtest-raf: ', acc_raf_raf)

###### FER
acc_raf_fer, test_loss = test(model, loader_fer, device)
print('test acc dbtrain-raf dbtest-fer: ', acc_raf_fer)

###### MMA
acc_raf_mma, test_loss = test(model, loader_mma, device)
print('test acc dbtrain-raf dbtest-mma: ', acc_raf_mma)

test acc dbtrain-raf dbtest-affect:  tensor(0.4740, device='cuda:0')
test acc dbtrain-raf dbtest-raf:  tensor(0.8941, device='cuda:0')
test acc dbtrain-raf dbtest-fer:  tensor(0.6214, device='cuda:0')
test acc dbtrain-raf dbtest-mma:  tensor(0.5554, device='cuda:0')
